# Coherence limit error of gates with three or more qubits (Qiskit Demoday 2022/07/21)

https://github.com/Qiskit/qiskit-experiments/pull/779

Toshinari Itoko

## What is coherence limit error
**The minimum $n$-qubit gate error achievable** on $n$ physical qubits with coherence limit.

= A value computed from $n$, `gate_length`, `T1` and `T2` values.

= **Average gate infidelity** (1 - average gate fidelity) of a noisy $n$-qubit gate with duration `gate_length`
- Assume **qubit-wise thermal relaxation error** (without excitation) is only the error source on a gate of interest
- Assume the thermal relaxation error approximately commutes with the gate (-> **gate independent** quantity)

Note: Thermal relaxation channel is parameterized by `T1` and `T2` values, which can be measured by experiments.

### Issue
- `RBUtils.coherence_limit` can compute only the coherence limit error of 1- or 2-qubit gates

In [1]:
from qiskit_experiments.library.randomized_benchmarking import RBUtils

In [12]:
# Run old function with num_qubits = 2
num_qubits = 2
t1s = [100 for _ in range(num_qubits)]
t2s = [100 for _ in range(num_qubits)]
gate_length = 5

RBUtils.coherence_limit(
    nQ=num_qubits,
    T1_list=t1s,
    T2_list=t2s,
    gatelen=gate_length
)

/var/folders/z0/psqdmn3d5pv75gs1dp30b0b00000gn/T/ipykernel_3616/1696815553.py:7: DeprecationWarning: The method 'coherence_limit' of 'RBUtils' class has been deprecated and will be removed after Qiskit Experiments 0.4. Please use coherence_limit_error function that can handle three or more qubits instead.
  RBUtils.coherence_limit(


0.057454334533604046

In [13]:
# Run old function with num_qubits = 3
num_qubits = 3
t1s = [100 for _ in range(num_qubits)]
t2s = [100 for _ in range(num_qubits)]
gate_length = 5

RBUtils.coherence_limit(
    nQ=num_qubits,
    T1_list=t1s,
    T2_list=t2s,
    gatelen=gate_length
)

/var/folders/z0/psqdmn3d5pv75gs1dp30b0b00000gn/T/ipykernel_3616/1602455470.py:7: DeprecationWarning: The method 'coherence_limit' of 'RBUtils' class has been deprecated and will be removed after Qiskit Experiments 0.4. Please use coherence_limit_error function that can handle three or more qubits instead.
  RBUtils.coherence_limit(


ValueError: Not a valid number of qubits

### What PR does
- Deprecate the original function `RBUtils.coherence_limit`
- Reimplement it as a new function `RBUtils.coherence_limit_error` from scratch


In [16]:
# Run new function with num_qubits = 2
num_qubits = 2
t1s = [100 for _ in range(num_qubits)]
t2s = [100 for _ in range(num_qubits)]
gate_length = 5

RBUtils.coherence_limit_error(
    num_qubits=num_qubits,
    gate_length=gate_length,
    t1s=t1s,
    t2s=t2s,
)

0.057454334533604094

In [19]:
import numpy as np

np.isclose(
    RBUtils.coherence_limit(
        nQ=num_qubits,
        T1_list=t1s,
        T2_list=t2s,
        gatelen=gate_length
    ),
    RBUtils.coherence_limit_error(
        num_qubits=num_qubits,
        gate_length=gate_length,
        t1s=t1s,
        t2s=t2s,
    ),
    15
)

/var/folders/z0/psqdmn3d5pv75gs1dp30b0b00000gn/T/ipykernel_3616/4074235615.py:4: DeprecationWarning: The method 'coherence_limit' of 'RBUtils' class has been deprecated and will be removed after Qiskit Experiments 0.4. Please use coherence_limit_error function that can handle three or more qubits instead.
  RBUtils.coherence_limit(


True

In [14]:
# Run new function with num_qubits = 3
num_qubits = 3
t1s = [100 for _ in range(num_qubits)]
t2s = [100 for _ in range(num_qubits)]
gate_length = 5

RBUtils.coherence_limit_error(
    num_qubits=num_qubits,
    gate_length=gate_length,
    t1s=t1s,
    t2s=t2s,
)

0.09401679901452938

In [15]:
# Run new function with num_qubits = 9
num_qubits = 9
t1s = [100 for _ in range(num_qubits)]
t2s = [100 for _ in range(num_qubits)]
gate_length = 5

RBUtils.coherence_limit_error(
    num_qubits=num_qubits,
    gate_length=gate_length,
    t1s=t1s,
    t2s=t2s,
)

0.2843733430854025

## Difference between original and new implementation

### Original implementation
https://github.com/Qiskit/qiskit-experiments/blob/c315cbd0062297b5a6696cf3e76d764833af303a/qiskit_experiments/library/randomized_benchmarking/rb_utils.py#L141

In [ ]:
def coherence_limit(nQ=2, T1_list=None, T2_list=None, gatelen=0.1):
    T1 = np.array(T1_list)

    if T2_list is None:
        T2 = 2 * T1
    else:
        T2 = np.array(T2_list)

    if len(T1) != nQ or len(T2) != nQ:
        raise ValueError("T1 and/or T2 not the right length")

    coherence_limit_err = 0

    if nQ == 1:

        coherence_limit_err = 0.5 * (
            1.0 - 2.0 / 3.0 * np.exp(-gatelen / T2[0]) - 1.0 / 3.0 * np.exp(-gatelen / T1[0])
        )

    elif nQ == 2:

        T1factor = 0
        T2factor = 0

        for i in range(2):
            T1factor += 1.0 / 15.0 * np.exp(-gatelen / T1[i])
            T2factor += (
                2.0
                / 15.0
                * (
                    np.exp(-gatelen / T2[i])
                    + np.exp(-gatelen * (1.0 / T2[i] + 1.0 / T1[1 - i]))
                )
            )

        T1factor += 1.0 / 15.0 * np.exp(-gatelen * np.sum(1 / T1))
        T2factor += 4.0 / 15.0 * np.exp(-gatelen * np.sum(1 / T2))

        coherence_limit_err = 0.75 * (1.0 - T1factor - T2factor)

    else:
        raise ValueError("Not a valid number of qubits")

    return coherence_limit_err

(Recap.) *Coherence limit error = Average gate infidelity at coherence limit*

## Equation behind the old implementation
$$
     \begin{align}
     1 - F_{\text{ave}}(\mathcal{E}, U)
            &= \frac{d-1}{d} \left(1 - \frac{Tr[{PTM}_{\Lambda}] - 1}{d^2-1}\right) \quad\mbox{(original)}
     \end{align}
$$
Note that the first diagonal element (the left-top corner element) of ${PTM}_{\Lambda}$ is always 1 for any thermal relaxation channel $\Lambda$.
The original code somehow computes the sum of the second to last diagonal elements of ${PTM}_{\Lambda}$  without explicitly using the formula $tr(X \otimes Y) = tr(X) tr(Y)$.

## Equation behind the new implementation
$$
     \begin{align}
     1 - F_{\text{ave}}(\mathcal{E}, U)
            &= \frac{d}{d+1} \left(1 - \frac{Tr[S_{\Lambda}]}{d^2}\right) \quad\mbox{(proposed)} \\
     \end{align}
$$
Note that $Tr[S_{\Lambda}] = Tr[{PTM}_{\Lambda}]$ where $S_{\Lambda}$ is the Liouville Superoperator and ${PTM}_{\Lambda}$ is the Pauli Transfer Matrix of a quantum channel $\Lambda$ because they can be transformed one another by basis transformation (unitary transformation),

### New implementation
https://github.com/Qiskit/qiskit-experiments/blob/73488cd367b6d095f3472a7fb7ec53ee32a15e1f/qiskit_experiments/library/randomized_benchmarking/rb_utils.py#L210

In [ ]:
def coherence_limit_error(
    num_qubits: int, gate_length: float, t1s: Sequence, t2s: Optional[Sequence] = None
):
     t1s = np.array(t1s)
    if t2s is None:
        t2s = 2 * t1s
    else:
        t2s = np.array([min(t2, 2 * t1) for t1, t2 in zip(t1s, t2s)])

    if len(t1s) != num_qubits or len(t2s) != num_qubits:
        raise ValueError("Length of t1s/t2s must equal num_qubits")

    def thermal_relaxation_choi(t1, t2, time):  # without excitation
        return qi.Choi(
            np.array(
                [
                    [1, 0, 0, np.exp(-time / t2)],
                    [0, 0, 0, 0],
                    [0, 0, 1 - np.exp(-time / t1), 0],
                    [np.exp(-time / t2), 0, 0, np.exp(-time / t1)],
                ]
            )
        )

    chois = [thermal_relaxation_choi(t1, t2, gate_length) for t1, t2 in zip(t1s, t2s)]
    traces = [np.real(np.trace(np.array(qi.SuperOp(choi)))) for choi in chois]
    d = 2**num_qubits
    return d / (d + 1) * (1 - functools.reduce(operator.mul, traces) / (d * d))

(Recap.)
$$
     \begin{align}
     1 - F_{\text{ave}}(\mathcal{E}, U)
            &= \frac{d}{d+1} \left(1 - \frac{Tr[S_{\Lambda}]}{d^2}\right) \quad\mbox{(proposed)} \\
     \end{align}
$$

# Discussion
The PR looks good but why not merged?
-> Got a good question from reviewers (Naoki and Chris).

- Where is the best place for the new `coherence_limit_error` function?

    - Somewhere else in `qiskit-experiments`?
    - `quantum_info` module in `qiskit-terra`?
    - `noise` module in `qiskit-aer`?

- Is the coherence limit error is widely accepted quantity?
    - If not, to be a sample code in a tutorial notebook?